# Первый вариант исследования состоял в подаче в pretrained DistilBertForSequenceClassification предложений корпуса из библейских и биологических **предложений**, где первым был присвоен лейбл 0, а вторым - лейбл 1. При этом тектсы не были очищены от стоп-слов.

### BIBLE


In [1]:
import numpy as np
import pandas as pd

In [2]:
bible = pd.read_csv('/content/prep_asb_SW.csv')


In [3]:
bible.head()

,Unnamed: 0,Text,cleaned_text,prep_text,len
0,0,In the beginning God created the heavens and t...,in the beginning god created the heavens and t...,in the beginning god created the heavens and t...,55
1,1,And the earth was waste and void; and darkness...,and the earth was waste and void and darkness...,and the earth was waste and void and darkness...,134
2,2,"And God said, Let there be light: and there wa...",and god said let there be light and there wa...,and god said let there be light and there wa...,54
3,3,"And God saw the light, that it was good: and G...",and god saw the light that it was good and g...,and god saw the light that it was good and g...,85
4,4,"And God called the light Day, and the darkness...",and god called the light day and the darkness...,and god called the light day and the darkness...,63


In [4]:
bible['prep_text'] = bible['prep_text'].astype(str)

In [5]:
def count_words(df):
    w = 0
    for idx, row in df.iterrows():
        w+= len(row['prep_text'].split())
    return w

print("Bible", count_words(bible))


Bible 789535


###BIOLOGY_ACADEMIC

In [9]:
bio = pd.read_csv('/content/prep_BIO_SW.csv')
bio.head()

,Unnamed: 0,text,cleaned_text,removed_terms_text,len
0,0,bs_bs_banner\n\nINVITED REVIEW SERIES:\nSTEM C...,bs bs banner invited review series stem cells ...,bs bs banner invited review series stem cells ...,140
1,1,IVAN BERTONCELLO AND JONATHAN L. MCQUALTER\n\n...,ivan bertoncello and jonathan mcqualter lung h...,and jonathan health research centre department...,494
2,2,"In this\nreview, we describe the attributes of...",in this review we describe the attributes of a...,in this review we describe the of adult stem a...,191
3,3,We describe the power and\nlimitations of expe...,we describe the power and limitations of exper...,we describe the power and of strategies and us...,214
4,4,The review summarizes\nrecent progress and obs...,the review summarizes recent progress and obst...,the review summarizes recent progress and in d...,161


In [10]:
bio['removed_terms_text'] = bio['removed_terms_text'].astype(str)

In [11]:
bible['target'] = 0
bio['target'] = 1

In [12]:
n = len(bible)
bio = bio.iloc[:n]
bible = bible.iloc[:n]

In [13]:
bio.rename({"removed_terms_text":"prep_text"}, axis=1, inplace=True) # переименуем колонку

In [14]:
bio['prep_text'] = bio['prep_text'].astype(str)

In [15]:
corpus = pd.concat([bible[['prep_text', 'target']], bio[['prep_text', 'target']]], axis=0).dropna()
corpus

,prep_text,target
0,in the beginning god created the heavens and t...,0
1,and the earth was waste and void and darkness...,0
2,and god said let there be light and there wa...,0
3,and god saw the light that it was good and g...,0
4,and god called the light day and the darkness...,0
...,...,...
35345,the of the primarily contend that and can disc...,1
35346,although the preserve their shape they are kno...,1
35347,meanwhile the of shape of argue that the of wi...,1
35348,therefore the interaction of the with and bind...,1


In [16]:
def clean_long_sent(df):
    df["len"] = df["prep_text"].apply(lambda x: len(x))
    return df[df.len <= 512]

corpus = clean_long_sent(corpus)
corpus

,prep_text,target,len
0,in the beginning god created the heavens and t...,0,55
1,and the earth was waste and void and darkness...,0,134
2,and god said let there be light and there wa...,0,54
3,and god saw the light that it was good and g...,0,85
4,and god called the light day and the darkness...,0,63
...,...,...,...
35345,the of the primarily contend that and can disc...,1,90
35346,although the preserve their shape they are kno...,1,92
35347,meanwhile the of shape of argue that the of wi...,1,99
35348,therefore the interaction of the with and bind...,1,123


# Обучение модели

In [17]:
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Device available for running: ")
print(device)

Device available for running: 
cuda


In [18]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW

model_checkpoint = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_checkpoint, padding=True, truncation=True)
model_tf = DistilBertForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
from sklearn.model_selection import train_test_split

X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(corpus['prep_text'],
                                                                corpus['target'],
                                                                test_size=0.2,
                                                                shuffle=True)

In [20]:
from torch.utils.data import TensorDataset, DataLoader

train_encodings = tokenizer(list(X_train_tf.values), padding=True)
test_encodings = tokenizer(list(X_test_tf.values), padding=True)

In [21]:
train_labels = torch.tensor(y_train_tf.values)
test_labels = torch.tensor(y_test_tf.values)


train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']),
                              torch.tensor(train_encodings['attention_mask']),
                              train_labels)
test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']),
                             torch.tensor(test_encodings['attention_mask']),
                             test_labels)

In [22]:
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [23]:
from tqdm import tqdm
from torch import nn
from torch.optim import Adam, AdamW

## Для того, чтобы ускорить процесс обучения модели, мы воспользовались библиотекой huggingface Accelerate: https://huggingface.co/docs/accelerate/index

In [25]:
! pip install accelerate
from accelerate import Accelerator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

Разница в использовании с Эксел:

In [26]:
accelerator = Accelerator(mixed_precision="fp16") # заводим акселлератор
optimizer_tf = AdamW(model_tf.parameters(), lr=2e-5) # заводим оптимайзер: класс, который обновляет веса

model, optimizer, train_loader = accelerator.prepare(model_tf, optimizer_tf, train_loader) # заворачиваем наши ... в оболочку акселлератора - удобный вид

model.train() # перевели модель в режим

epochs = 5
criterion = nn.CrossEntropyLoss() # наш лосс - функция потерь, которую мы минимизируем ф-ция потерь, используемая для классификации
for epoch in tqdm(range(epochs)): # для каждой модели
    epoch_loss = 0 # лосс на каждой эпохе выводим
    for i, batch in enumerate(train_loader): # элемент трейн лоудера это число предложений, равное размеру батча
        optimizer.zero_grad() # градиент класс, занимающийся обновлением весов модели: нужен из-за конструкции торча
        texts, atts, labels = batch[0].to(device), batch[1].to(device), batch[2].to(device) # распаковываем батч
        output = model(texts, attention_mask=atts) # подаем тексты и этт маски

        loss = criterion(output.logits, labels) # считаем лосс - вероятности принадлености к первому классу
        accelerator.backward(loss) # акселлератор берет на себя функцию бэкворда: подсчет градиентов
        epoch_loss += loss # выводим лосс за всю эпоху
        optimizer.step() # на этом шаге происходит обновление весов и м можем сделать след шаг - переход к след батчу
    print(f"Train loss: {epoch_loss}")

 20%|██        | 1/5 [10:08<40:34, 608.52s/it]

Train loss: 140.4393768310547


 40%|████      | 2/5 [20:12<30:17, 605.87s/it]

Train loss: 76.78878784179688


 60%|██████    | 3/5 [30:17<20:10, 605.29s/it]

Train loss: 71.66419982910156


 80%|████████  | 4/5 [40:22<10:05, 605.29s/it]

Train loss: 64.36109161376953


100%|██████████| 5/5 [50:29<00:00, 605.86s/it]

Train loss: 63.961124420166016


In [ ]:

# Это наш оригинальный цикл до использоания Accelerator
# epochs = 10
# criterion = nn.CrossEntropyLoss()
# for epoch in tqdm(range(epochs)):
#     model_tf.train()
#     epoch_loss = 0
#     for i, batch in enumerate(train_loader):
#         optimizer_tf.zero_grad()
#         texts, atts, labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
#         output = model_tf(texts, attention_mask=atts)

#         loss = criterion(output.logits, labels) -
#         loss.backward()
#         epoch_loss += loss
#         optimizer_tf.step()
#     print(f"Train loss: {epoch_loss}")

In [27]:
predictions = []
probabilities = [] # наши логгиты - вероятности принадлежности к двум классам
original_labels = []

with torch.no_grad():
    model.eval()
    for batch in test_loader:
        texts, atts, labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
        output = model(texts, attention_mask=atts)
        probabilities.extend(output.logits.cpu()) # матрица, 2 колонки и строчек, сколько в батче
        predictions.extend(output.logits.argmax(axis=1).cpu()) # смотрим для каждого эл-та батча, достаем аргумент максимума для определения, к какому лейблу относится
        original_labels.extend(labels.cpu())

In [28]:
from sklearn.metrics import classification_report

print(classification_report(original_labels, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7053
           1       1.00      1.00      1.00      7087

    accuracy                           1.00     14140
   macro avg       1.00      1.00      1.00     14140
weighted avg       1.00      1.00      1.00     14140



In [29]:
torch.save(model.state_dict(), '/content/sentence_bert.pth')



# Смотрим, как модель определяет библейские цитаты

In [49]:
econ = pd.read_csv("/content/prep_bio_Economist_SW.csv")
econ

,Unnamed: 0,Text,cleaned_text,len
0,0,FOR THE past four billion years or so the only...,for the past four billion years or so the only...,154
1,1,Sometimes the gene would be damaged or scrambl...,sometimes the gene would be damaged or scrambl...,96
2,2,From that raw material arose the glories of na...,from that raw material arose the glories of na...,62
3,3,"But beneath it all, gene begat gene.That is no...",but beneath it all gene begat gene that is no ...,58
4,4,Now genes can be written from scratch and edit...,now genes can be written from scratch and edit...,90
...,...,...,...,...
35847,35972,"He found that, after 15 days, parasite loads i...",he found that after days parasite loads in ani...,269
35848,35973,Dr Kumar plans to try—though he will use genet...,dr kumar plans to try though he will use genet...,137
35849,35974,"But many, probably most, doctors are suspiciou...",but many probably most doctors are suspicious ...,165
35850,35975,"Dr Kumar’s findings are, nevertheless, interes...",dr kumar s findings are nevertheless interesting,49


In [55]:
econ['cleaned_text'] = econ['cleaned_text'].astype(str)

In [56]:
def clean_long_sent(df):
    df["len"] = df["cleaned_text"].apply(lambda x: len(x))
    return df[df.len <= 512]

cleaned_df = clean_long_sent(econ)

In [57]:
infer_encodings = tokenizer(list(cleaned_df["cleaned_text"].values), padding=True)
infer_dataset = TensorDataset(torch.tensor(infer_encodings['input_ids']),
                              torch.tensor(infer_encodings['attention_mask']))

infer_loader = DataLoader(infer_dataset, batch_size=16, shuffle=False)

predictions = []
probabilities = []

with torch.no_grad():
    model.eval()
    for batch in infer_loader:
        texts, atts = batch[0].to(device), batch[1].to(device)
        output = model(texts, attention_mask=atts)
        probabilities.extend(output.logits.cpu())
        predictions.extend(output.logits.argmax(axis=1).cpu())

In [58]:
predictions

[tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 ten

In [59]:
np.array(predictions) == 0

array([False, False, False, ..., False, False, False])

In [60]:
predictions = np.array(predictions)

In [61]:
cleaned_df[predictions == 0].to_csv('/content/sentence_result.csv')

In [64]:
import numpy as np
import pandas as pd

In [83]:
Res = pd.read_csv('/content/sentence_result.csv')
Res

,Unnamed: 0.1,Unnamed: 0,Text,cleaned_text,len
0,12,12,"But look back through history, and humanity’s ...",but look back through history and humanity s r...,306
1,13,13,"All brought prosperity and progress, but with ...",all brought prosperity and progress but with d...,67
2,17,17,"But much wilderness has been lost, and carbon ...",but much wilderness has been lost and carbon a...,204
3,20,20,This week Burger King introduced into some of ...,this week burger king introduced into some of ...,219
4,23,23,"Horses, cattle and cotton were introduced to t...",horses cattle and cotton were introduced to th...,117
...,...,...,...,...,...
6706,35815,35940,Throw them into a landfill and they will be go...,throw them into a landfill and they will be go...,58
6707,35817,35942,Their manufacturers hope that charging a “gree...,their manufacturers hope that charging a green...,86
6708,35822,35947,How green will be my valley?,how green will be my valley,28
6709,35844,35969,"Moreover, they produced 0.8 milligrams of arte...",moreover they produced milligrams of artemisin...,201


In [85]:
Res = Res.drop('Unnamed: 0.1', axis=1)

In [86]:
Res = Res.drop('Unnamed: 0', axis=1)

In [87]:
Res

,Text,cleaned_text,len
0,"But look back through history, and humanity’s ...",but look back through history and humanity s r...,306
1,"All brought prosperity and progress, but with ...",all brought prosperity and progress but with d...,67
2,"But much wilderness has been lost, and carbon ...",but much wilderness has been lost and carbon a...,204
3,This week Burger King introduced into some of ...,this week burger king introduced into some of ...,219
4,"Horses, cattle and cotton were introduced to t...",horses cattle and cotton were introduced to th...,117
...,...,...,...
6706,Throw them into a landfill and they will be go...,throw them into a landfill and they will be go...,58
6707,Their manufacturers hope that charging a “gree...,their manufacturers hope that charging a green...,86
6708,How green will be my valley?,how green will be my valley,28
6709,"Moreover, they produced 0.8 milligrams of arte...",moreover they produced milligrams of artemisin...,201


In [94]:
Res['Text'] = Res['Text'].astype(str)

In [91]:
Res.iloc[250:300, 0:1]

,Text
250,It also showed that this distinctive sound ind...
251,"From others, they took the ninth primaries."
252,And from others still they took the sevenths.
253,Each bird was then released and the sound of i...
254,When the recordings of the departing birds wer...
255,"In other words, it is the part of the whistle ..."
256,"AS A human, you are undoubtedly an animal, eve..."
257,"Like other animals, you eat, breathe and mate."
258,It is only 5m years since your ancestors parte...
259,But once into his stride there is no stopping ...


In [96]:
Res.loc[0].Text

'But look back through history, and humanity’s relations with the living world have seen three great transformations: the exploitation of fossil fuels, the globalisation of the world’s ecosystems after the European conquest of the Americas, and the domestication of crops and animals at the dawn of agriculture.'

In [97]:
Res.iloc[3].Text

'This week Burger King introduced into some of its restaurants a beefless Whopper that gets its meatiness from an engineered plant protein; such innovations could greatly ease a shift to less environmentally taxing diets.'

In [99]:
Res.iloc[25].Text

'But the very fact that it is happening at all is, surely, testament to an idea whose hour has come.'

**John 5:25**
American Standard Version

***Verily, verily, I say unto you, The hour cometh, and now is, when the dead shall hear the voice of the Son of God; and they that hear shall live.***

In [100]:
Res.iloc[5].Text

'Measles, smallpox and other pathogens ran through the New World like a forest fire, claiming tens of millions of lives.'

In [101]:
Res.iloc[9].Text

'This time, there will be foresight.'

***prophesy***

In [102]:
Res.iloc[22].Text

'They pick up others during the messy process of birth, and more from their mother’s milk.'

In [103]:
Res.iloc[27].Text

'It will be the first commercial product made in this way.'

In [104]:
Res.iloc[119].Text

'In a small gesture in this direction, Ginkgo has made a scent that smells of a type of hibiscus that is now extinct.This idea, too, meets with scepticism, even repugnance.'

In [105]:
Res.iloc[248].Text

'In the years that have passed since his death, ornithologists have proved time and again that birds’ songs, squawks and shrieks are used for sending signals to their kin, their rivals and sometimes even their predators.'

In [145]:
Res.iloc[876].Text

'Asked if he had a message to send to Mr Trump, he replied, “His view about ecology…he does not consider it important, and with that I disagree.” The Tibetan spiritual leader added, “Now I think America is learning lessons on the importance of ecology…on the east coast, floods, and on the west coast, [forest] fires.'

In [146]:
Res.iloc[5678].Text

'They found that the animals could receive sufficient amounts of methionine without having to rely upon synthesised sources, and they found that birds showed no obvious change in appearance or health.'